In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.python import keras
from tensorflow.python.keras.layers import Dense
import matplotlib.pyplot as plt

In [ ]:
# 获取训练数据与测试数据
mnist_train_data = '../../dataset/mnist_data/mnist/mnist_train/'
def get_data(data_dir):
    file_list = os.listdir(data_dir)
    np.random.shuffle(file_list)
    datas = []
    labels = []
    for file in file_list:
        files = file.split('.')
        if files[-1] == 'jpg':
            image = np.array(Image.open(data_dir+file))
            image = image.flatten() # 将28*28的图片转换为784的向量
            image = image/255.
            datas.append(image)
            labels.append(int(files[0]))
    # 将list转换为数组
    datas = np.array(datas)
    labels = np.array(labels)
    return datas,labels

In [ ]:
x_train,y_train = get_data(mnist_train_data)

In [ ]:
# 编码器模型
encoder = keras.Sequential([
    Dense(256,activation='relu'),
    Dense(64,activation='relu'),
    Dense(16,activation='relu'),
    Dense(2)
])

In [ ]:
# 解码器模型
decoder = keras.Sequential([
    Dense(16,activation='relu'),
    Dense(64,activation='relu'),
    Dense(256,activation='relu'),
    Dense(784,activation='sigmoid')
])

In [ ]:
# 自编码器与训练
autoencoder = keras.Sequential([
    encoder,
    decoder
])
# compile
autoencoder.compile(optimizer='adam',loss=keras.losses.binary_crossentropy)
autoencoder.fit(x_train,y_train,epochs=20,batch_size=256)

In [ ]:
# 图像重构测试
example_to_show = 10
rec = autoencoder.predict(x_train[:example_to_show])
f,ax = plt.subplots(2,10,figsize=(10,2))
for i in range(example_to_show):
    ax[0][i].imshow(np.reshape(x_train[i],(28,28)))
    ax[1][i].imshow(np.reshape(rec[i],(28,28)))
plt.show()

In [ ]:
# 图像分类测试
batch_x = x_train[:1000]
batch_y = y_train[:1000]
encoded = encoder.predict(batch_x)
plt.scatter(encoded[:, 0], encoded[:, 1], c=batch_y)
plt.show()